In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html
import time
from datetime import datetime
from unidecode import unidecode
import warnings
import time
from tqdm import tqdm
import os

import crawler
import models

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.49"}

def ObtainAllUrls():
    marcas = sorted([
        'mazda',
        'mercedes-benz','nissan','bmw',
        'chevrolet','toyota','renault','audi',
        'volkswagen','honda','kia','ford'])
    # marcas = sorted([
    # 'subaru','mazda','mahindra','mini','dodge','citroen','mitsubishi','hyundai','ssangyong','mg',
    # 'mercedes-benz','nissan','bmw','chevrolet','toyota','jeep','ds','dfm/dfsk','unico-dueno-nissan',
    # 'brilliance','mercedes-benz-blindada','renault','audi','chery','foton','zotye','seat','hafei',
    # 'skoda','porsche','opel','changan','volkswagen','fiat','ram','suzuki','honda','great-wall','kia',
    # 'jac','cupra','volvo','jaguar','peugeot','land-rover','hummer','daihatsu','byd','ford'])

    dim_ubicacion = pd.read_csv("dim_ubicacion_all.csv", sep="|")
    dim_ubicacion = dim_ubicacion.sample(3)
    #dim_ubicacion = dim_ubicacion[dim_ubicacion["ciudad"] == "cali"]
    oferts_full = []

    departamentos = dim_ubicacion["departamento"].unique().tolist()
    for departamento in tqdm(departamentos, desc="Parseando ofertas de paginación"):
        #print(departamento)

        ciudades = dim_ubicacion[dim_ubicacion["departamento"] == departamento]["ciudad"].unique().tolist()
        for ciudad in ciudades:
            #print(ciudad)
            
            for marca in marcas:
                size = 0 # validacion
                for pag in range(49, 1969, 48):  # 48*n | El máximo de paginación por categoría es de 42, 1969 vehículos
                    url = f"https://carros.mercadolibre.com.co/{marca}/{departamento}/{ciudad}/_Desde_{pag}_ITEM*CONDITION_2230581_NoIndex_True"
                    response = requests.get(url, headers=headers)
                    content = response.content.decode('utf-8')
                    href_ofert = '//a[contains(@class, "poly-component__title")]'
                    
                    if "Escribe en el buscador lo que quieres encontrar." not in content:
                        tree = html.fromstring(content)
                        urls_ofert_temp = [x.get('href') for x in tree.xpath(href_ofert)]
                        oferts_full.append(urls_ofert_temp)
                        time.sleep(1)
                        size += len(urls_ofert_temp)
                        #print(url)
                    else:
                        base_url = "/".join(url.split("/")[:6])
                        response = requests.get(base_url, headers=headers)
                        tree = html.fromstring(response.content)
                        urls_ofert_temp = [x.get('href') for x in tree.xpath(href_ofert)]
                        oferts_full.append(urls_ofert_temp)
                        time.sleep(1)
                        size += len(urls_ofert_temp)
                        #print(base_url)
                        
                        break
                    
                #print(marca, size)

    oferts_full = [i for sublist in oferts_full for i in sublist]
    oferts_full = list(set(oferts_full))
    pd.DataFrame({"url":oferts_full}).to_parquet("urls.parquet", compression = "gzip")
    print("Total de ofertas:", len(oferts_full))

    return oferts_full


def ReadAllUrls():
    oferts_full = pd.read_parquet("urls.parquet")["url"].tolist()
    oferts_full = [i for sublist in oferts_full for i in sublist]
    oferts_full = list(set(oferts_full))
    print(len(oferts_full))


def ScrapUrl(oferts_full, 
             delay = 1,
             counter_file = 200,
             repetead = False
             ):
    df = pd.DataFrame()
    error_server = 0
    counter = 1
    record_count = 0
    repetead = "_R" if repetead == True else "C" # C: registros correcto, R: registros que se volvieron a ingestar debido a error en andes-table

    actual_date = datetime.now()
    month = str(actual_date.month)
    month = np.where(len(month) == 1, "0" + month, month)
    day = str(actual_date.day)
    day = np.where(len(day) == 1, "0" + day, day)
    actual_date = str(actual_date.year) +  str(month) + str(day)

    for oferta_temp in tqdm(list(oferts_full), desc="Obteniendo variables"):
        df_oferta_temp, error_server_temp = crawler.GetCarAtributes(oferta_temp, debug = "OFF")
        df_oferta_temp = df_oferta_temp[df_oferta_temp["estado"] == 1]
        df = pd.concat([df, df_oferta_temp], axis = 0)

        # if "marca" in df_oferta_temp.columns:
        #     print("correcto")
        # else:
        #     print("incorrecto")

        error_server += error_server_temp
        if error_server > len(oferts_full)*0.3:
            print("WARNING: Error Server > 30% registers")
            break

        
        #if "marca" in df_oferta_temp:

        record_count += 1
        if record_count == counter_file:
            df.to_csv(f"temp/{actual_date}_{str(counter) + repetead}.csv", sep="|", index=False)
            df = pd.DataFrame()
            counter += 1
            record_count = 0  
            time.sleep(15)

        time.sleep(delay)
    print("Oferts Error Server 403:", error_server)
    print("Dimensión df:", df.shape)
    return df


def ReadTemp():
    files = os.listdir("temp")
    files = [pd.read_csv("temp/" + x, sep = "|") for x in files]
    df = pd.concat(files, axis = 0)
    df.drop_duplicates(inplace=True)
    print("Dimension ./temp:", df.shape)
    return df


def TrainModel(df):
    cols_numeric = ["precio", "año", "km", "motor", "km_por_año"]
    for col in cols_numeric:
        df[col] = df[col].astype(float)
    df_train = df[["precio", "año", "km", "motor", "km_por_año", "transmision", "tipo_de_combustible", "tipo_de_carroceria", "puertas",
                   "marca", "modelo_agrup"]].dropna()
    df_train["precio"] = np.log(df_train["precio"].astype(float))

    X_train, X_test, y_train, y_test = models.TrainTestDummies(df = df_train, y='precio',
                                                                 dummies = ["marca", "modelo_agrup", "transmision", "tipo_de_combustible", 
                                                                            "tipo_de_carroceria", "puertas"])
    print("Dimensión train:", X_train.shape)
    print("Dimensión test:", X_test.shape)

    param_grid = {
        'n_estimators': [380],
        'max_depth': [5],
        'learning_rate': [0.2],
        'min_child_weight': [5],
        'booster': ['gbtree'],
        'eta': [0.001],
        'gamma': [0.001],
        'lambda': [0.99],
        'alpha': [0.99],
        'subsample': [1],
        'colsample_bytree': [0.5],
        'objective': ['reg:squarederror']
    }
    model = models.XGBoost(X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test, 
                        param_grid = param_grid, cv = 2).fit()

    models.SaveModel(model, "xgboost.pkl")

    return model

ModuleNotFoundError: No module named 'unidecode'

In [3]:
!pip install pandas
!pip install scikit-learn
!pip install unidecode
!pip install matplotlib 
!pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/235.5 kB ? eta -:--:--
   -------------------------------------- - 225.3/235.5 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 235.5/235.5 kB 2.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Ejecucion

In [ ]:
print("Iniciando Web Scrapping")
counter_file = 100
delay = 1

time1 = datetime.now()

#func.ClearFolder("temp")

# Obtener número actual de ofertas
#primer_pagina = "https://carros.mercadolibre.com.co/valle-del-cauca/cali/_Desde_49_ITEM*CONDITION_2230581_NoIndex_True"
primer_pagina = "https://carros.mercadolibre.com.co/_Desde_49_ITEM*CONDITION_2230581_NoIndex_True" # Nacional
response = requests.get(primer_pagina, headers=headers)
response.raise_for_status()
tree = html.fromstring(response.content)
num_ofertas = int(tree.xpath('//*[@id="root-app"]/div/div[3]/aside/div[2]/span')[0].text_content().strip().replace(".","").replace(" resultados",""))
print("Total ofertas:", num_ofertas)
print("="*120)


print("Bloque 1: obtener la url de cada oferta")
oferts_urls = ObtainAllUrls()
#oferts_urls = pd.Series(oferts_urls).sample(200).tolist()
print("="*120)

oferts_urls = pd.Series(oferts_urls).sample(3000).tolist()


print("Bloque 2: obtener los atributos de cada oferta")
df = ScrapUrl(oferts_urls, delay = delay, counter_file = counter_file)
print("="*120)


print("Bloque 3: volver a consumir los registros con errores")
df = ReadTemp()
urls_complete = df[df["estado"] == 1]["url"]
pendient = set(oferts_urls) - set(urls_complete)
print("Registros correctos:", len(urls_complete), round(len(urls_complete)/(len(urls_complete) + len(pendient))*100), 2)
print("Registros con error:", len(pendient), round(len(pendient)/(len(urls_complete) + len(pendient))*100), 2)
df_error = ScrapUrl(pendient, delay = delay, counter_file = counter_file, repetead = True)


print("Bloque 4: concatenado final de archivos")
df = ReadTemp()
df = df[df["estado"] == 1]
print("="*120)


print("Bloque 5: refinando atributos")
df = crawler.RefineAtributes(df)
print("-"*120)

print("Correctos final:", df[df["estado"] == 1].shape[0], "|", round((df[df["estado"] == 1].shape[0]/df.shape[0])*100, 2), "%")
print("Dimensión df:", df.shape)
vars_duplicates = ["marca", "modelo", "precio", "año", "motor", "km", "transmision", "tipo_de_carroceria", "tipo_de_combustible", 
                   "barrio", "ciudad", "departamento"]
print("Duplicados por atributos:", df[df.duplicated(vars_duplicates)].shape[0])
print("Duplicados por url:", df[df.duplicated(["url"])].shape[0])
print("Duplicados por id:", df[df.duplicated(["id_ofert"])].shape[0])
df = df.drop_duplicates("url").drop_duplicates("id_ofert").drop_duplicates(vars_duplicates)
print("Dimensión df sin duplicados:", df.shape)
print("-"*120)
pendient = set(oferts_urls) - set(df["url"])
print("Final: Registros correctos:", len(urls_complete))
print("Final: Registros con error (excluidos):", len(pendient))
print("="*120)


print("Bloque 6: imputación de datos faltantes en categoricas"); print("-"*50)
df = models.ImputCategorics(df, "marca_modelo", "transmision", 70)
df = models.ImputCategorics(df, "marca_modelo", "tipo_de_carroceria", 70)
df = models.ImputCategorics(df, "marca_modelo", "tipo_de_combustible", 70)
df = models.ImputCategorics(df, "marca_modelo", "puertas", 70)
print("="*120)


print("Bloque 7: criterios de exclusión e inclusión"); print("-"*50)
dim = df.shape[0]
print("Registros:", dim)
print("Excluyendo...")
umbral_precio = np.percentile(df["precio"], 97)
df = df[(df["precio"] > 0) & (df["precio"] <= umbral_precio)]
print(f"Precio != 0 & < {round(umbral_precio,2)}:", df.shape)
df = df[df["año"] != int(datetime.today().year)]
print("Año != actual:", df.shape)
df = df[df["km"] != 0]
print("Km > 0:", df.shape)
df = df[(df["motor"] > 0.7) | (df["motor"].isnull())]
print("Motor > 0.7 | null", df.shape)
df = df[df["antiguedad"] > 0]
print("Antigúedad > 0:", df.shape)
df = df[~df["tipo_de_carroceria"].isin(["Eléctrico", 'Light Truck','Monovolumen', 'ESTACAS'])]
print("Carrocerias:", df.shape)
df = df[df["transmision"].notna()]
print("transmision notna:", df.shape[0])
df = df[df["tipo_de_combustible"].notna()]
print("Combustible notna:", df.shape[0])
df = df[df["tipo_de_carroceria"].notna()]
print("Carroceria notna:", df.shape[0])
print("*Total eliminados:", dim - df.shape[0], f"({round(((dim - df.shape[0]) / dim)*100, 2)}%)")
print("="*120)


print("Bloque 8: imputando valores faltantes en cilindraje"); print("-"*50)
df["motor"] = np.log(df["motor"])
df = models.ImputRegression(df, y = 'motor', x = ['año', 'marca', 'modelo', 'tipo_de_combustible', 'transmision', 'tipo_de_carroceria', 
                                                  'puertas'], method = "xgboost")
df["motor"] = round(np.exp(df["motor"]), 2)
print("="*120)


print("---------- Precio ----------")
print("Media:", round(df["precio"].mean(), 2))
print("Mediana:", round(df["precio"].median(), 2))
print("CV:", round(np.sqrt(df["precio"].var()) / df["precio"].mean() * 100, 2), "%")
print("Umbral:", round(umbral_precio, 3))
print("="*120)


print("Bloque 9: agrupacion de variables"); print("-"*50)
# df = models.ClassicationTreeGroup(df, y = "precio", x = "marca", max_leaf_nodes=[4, 5, 6]); print("-"*50)
df = models.ClassicationTreeGroup(df, y = "precio", x = "modelo", max_leaf_nodes=[30])
print("="*120)

df = df[['estado', 'nombre', 'precio', 'año', 'km', 'km_por_año', 'barrio', 'ciudad','departamento', 'fecha', 'url', 'dtm_etl', 
         'marca_modelo', 'marca', 
         'modelo', 'modelo_agrup', 'version','color', 'tipo_de_combustible', 'puertas', 'transmision', 'motor','tipo_de_carroceria', 
         'ultimo_digito_de_la_placa','con_camara_de_reversa']]

df.to_csv("df_refine.csv", sep = "|", index = False)


print("Bloque 10: entrenando modelo predictivo"); print("-"*50)
model = TrainModel(df)

print("Dimensión final:", df.shape)
print(f"El proceso tardó: {datetime.now() - time1}")
print("="*120)

## Predicciones

In [ ]:
cols_numeric = ["precio", "año", "km", "motor", "km_por_año"]
for col in cols_numeric:
    df[col] = df[col].astype(float)
df_train = df[["precio", "año", "km", "motor", "km_por_año", "transmision", "tipo_de_combustible", "tipo_de_carroceria", "puertas",
                "marca", "modelo_agrup"]].dropna()
df_train["precio"] = np.log(df_train["precio"].astype(float))

X_train, X_test, y_train, y_test = models.TrainTestDummies(df = df_train, y='precio',
                                                                dummies = ["marca", "modelo_agrup", "transmision", "tipo_de_combustible", 
                                                                        "tipo_de_carroceria", "puertas"])
print("Dimensión train:", X_train.shape)
print("Dimensión test:", X_test.shape)

In [ ]:
año = 2005
km = 150000
motor = 4
marca = "Ford"
modelo = "Explorer"
tipo_de_carroceria = "Camioneta"
puertas = "4_5"
transmision = "Automático"
tipo_de_combustible = "Gasolina"
modelo = df[df["modelo"] == modelo]["modelo_agrup"].unique()[0]
df_new = pd.DataFrame({"año":[año], "km":[km], "motor":[motor], "km_por_año":[km/(2024-año)], 
                       "transmision":[transmision], "tipo_de_combustible":[tipo_de_combustible],
                       "marca":[marca], "modelo_agrup":[modelo], "puertas":[puertas], 
                       "tipo_de_carroceria":[tipo_de_carroceria]})
df_new = pd.get_dummies(df_new, columns = ["transmision", "tipo_de_combustible", "marca", "modelo_agrup",
                                           "puertas", "tipo_de_carroceria"]).replace(True, 1)

cols = X_train.columns.tolist()
for col in cols:
    if col not in df_new.columns:
        df_new[col] = 0
df_new = df_new[X_train.columns.tolist()]
np.exp(model.predict(df_new))

## Utils

In [ ]:
df_ubicacion = df[["ciudad", "departamento"]].drop_duplicates().sort_values("departamento").dropna()
df_ubicacion["ciudad"] = [unidecode(x.lower().rstrip(" ").lstrip(" ").replace(" ", "-")) for x in df_ubicacion["ciudad"].astype(str)]
df_ubicacion["departamendf_ubicacion.to_csv("dim_ubicacion.csv", sep = "|", index = False)to"] = [unidecode(x.lower().rstrip(" ").lstrip(" ").replace(" ", "-")) for x in df_ubicacion["departamento"].astype(str)]
df_ubicacion["departamento"] = df_ubicacion["departamento"].replace({"bogota-d.c.":"bogota-dc"})

In [ ]:
df_ubicacion.to_csv("dim_ubicacion_full.csv", sep = "|", index = False)

In [ ]:
len(df_ubicacion["ciudad"].unique())

## Auxiliares

In [ ]:
marcas = set([unidecode(x.lower()).replace(" ", "-") for x in df["marca"]])
print(len(marcas))
"','".join(marcas)